In [11]:
library(statsecol)
library(jagsUI)
library(MCMCvis)
library(ggplot2)
data("voles")

Warning message:
“package ‘jagsUI’ was built under R version 4.1.1”
Warning message:
“package ‘ggplot2’ was built under R version 4.1.1”


In [ ]:
sink("voles.txt")
cat("
model{
  # Prior specification
  psi1 ~ dunif(0, 1)
  mu ~ dnorm(0, 0.001)
  sigma ~ dunif(0, 100)
  b0 ~ dnorm(0, 0.001)
  b1 ~ dnorm(0, 0.001)
  a0 ~ dnorm(0, 0.001)
  a1 ~ dnorm(0, 0.001)

  for(site in 1:n.sites) {
    for(time in 1:(n.years - 1)) {
      l.gamma[site, time] = b0 + b1 * connectivity[site]
      logit(gamma[site, time]) = l.gamma[site, time]
      l.epsilon[site, time] = a0 + a1 * length[site]
      logit(epsilon[site, time]) = l.epsilon[site, time]
    }
  }
   
  # Likelihood
  for(site in 1:n.sites){
    z[site, 1] ~ dbern(psi1)
    for(year in 2:n.years) {      
      z[site, year] ~ dbern(
        (1 - z[site, year - 1]) * gamma[site, year - 1] + z[site, year - 1] * (1 - epsilon[site, year - 1])
      )
    }
  }

  for(site in 1:n.sites) {
    for(year in 1:n.years) {
      q[site, year] ~ dnorm(mu, sigma^(-2))
      logit(p[site, year]) = q[site, year]
      y[site, year] ~ dbin(p[site, year] * z[site, year], visits[site, year])
    }
  }
  
  for(year in 1:n.years){
    propocc[year] <- sum(z[, year]) / n.sites   
  }
}
",fill = TRUE)
sink()

In [41]:
visits = as.matrix(voles[,c("j1", "j2", "j3", "j4")])
y = as.matrix(voles[, c("y1", "y2", "y3", "y4")])

In [51]:
# Bundle data
volesdata <- list(
    y = y,
    n.years = 4,
    n.sites = nrow(voles),
    visits = visits,
    connectivity = voles$Connectivity,
    length = voles$Length
)

volesinits <- function() {
  list(psi1 = runif(1, 0, 1),
       mu = runif(1, 0, 5),
       sigma = runif(1, 0, 1),
       b0 = runif(1, -2, 2),
       b1 = runif(1, -2, 2),
       a0 = runif(1, -2, 2),
       a1 = runif(1, -2, 2))
}

# Parameters monitored
volesparams <- c("gamma", "epsilon", "p", "propocc", 
                 "Chi2.obs", "Chi2.sim", "FT.obs", "FT.sim")

# MCMC settings
ni <- 5000
nt <- 4
nb <- 1000
nc <- 3

volesout <- jags(
    data = volesdata,
    inits = volesinits,
    parameters.to.save = volesparams,
    model.file = "voles.txt",
    n.chains = nc,
    n.iter = ni,
    n.burnin = nb,
    n.thin = nt
)


Processing function input....... 

Done. 
 
Compiling model graph
   Resolving undeclared variables
Deleting model



ERROR: Error in jags.model(file = model.file, data = data, inits = inits, n.chains = n.chains, : RUNTIME ERROR:
Compilation error on line 34.
Unknown variable time
Either supply values for this variable with the data
or define it  on the left hand side of a relation.




In [49]:
nrow(voles)

[1] 114